<a href="https://colab.research.google.com/github/HazelvdW/context-framed-listening/blob/main/NLP_framed_listening.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP analysis for Framed Listening study.
> Authored by **Hazel A. van der Walle** (PhD student, Music, Durham University), September 2025.

All datasets generated and used for this study are openly available on GitHub https://github.com/HazelvdW/context-framed-listening.

The cleaned raw data (processed in R) are used in this notebook, so let's clone necessary files and directories:

In [1]:
!git clone https://github.com/HazelvdW/context-framed-listening.git

Cloning into 'context-framed-listening'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 28 (delta 12), reused 15 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (28/28), 255.16 KiB | 3.64 MiB/s, done.
Resolving deltas: 100% (12/12), done.


You should now have a file called **"context-framed-listening"** in this notebook.

< Check this out by clicking on the folder icon on the lefthand side panel in this webpage (press the refresh symbol if you can't see it yet).

For this NLP analysis, we are only working from the file **"data_study1_MAIN.csv"** which contains participants qualitative thought descriptions.





---
## Setup

Start by importing the necessary packages in the cell below:

In [2]:
import os
import csv
import pandas as pd
import numpy as np

Load in the data .csv file:

In [3]:
data = pd.read_csv("/content/context-framed-listening/data_study1_MAIN.csv")

This dataset contains every participant's response to all 16 clip-context stimuli pairings.

_For the purposes of this analysis_ we are only interested in trials where music-evoked thoughts (METs) were expereienced and described – this is all rows where "descr_THOUGHT.text" is _not_ NA.

Let's create a new dataset that only contains trials with METs:

In [4]:
dataMET = data[data['descr_THOUGHT.text'].notna()].copy()

Familiarise yourself with the data structure by taking a quick look through before we dig into any analyses.

In [5]:
display(dataMET)

# print out all column headers so we have a quick copy for later reference
print(dataMET.columns)

,clip_name,context_word,expName,PROLIFIC_PID,File_ID,date,response_thought_or_not.keys,descr_THOUGHT.text,rating_music_prompted.response,rating_spontaneity.response,...,demographics.livingCountry,demographics.birthCountry,demographics.nativeLanguage,demographics.otherLanguage,demographics.otherLanguageText,demographics.hearingImpariments,demographics.hearingImpairmentsText,demographics.education,demographics.musicianIdentification,demographics.feedback
0,80s_LOW_02_Breaking_Away.mp3,bar,clip_context_g1,5eff5f05b92981000a2aed73,clip_context_g1_5eff5f05b92981000a2aed73_02059...,2025-07-01_10h45.13.126,y,"kind of sad, melancholy. not happy or upbeat. ...",5.0,4.0,...,United Kingdom,United Kingdom,English,False,NaN,False,NaN,5,2,NaN
1,Jazz_MED_07_Turiya_and_Ramakrishna.mp3,video game,clip_context_g1,5eff5f05b92981000a2aed73,clip_context_g1_5eff5f05b92981000a2aed73_02059...,2025-07-01_10h45.13.126,y,it did not sound like a video game. if anythin...,5.0,5.0,...,United Kingdom,United Kingdom,English,False,NaN,False,NaN,5,2,NaN
2,80s_MED_08_After_Tonight.mp3,video game,clip_context_g1,5eff5f05b92981000a2aed73,clip_context_g1_5eff5f05b92981000a2aed73_02059...,2025-07-01_10h45.13.126,y,"overly upbeat. no real emotions, peppy. too mu...",5.0,4.0,...,United Kingdom,United Kingdom,English,False,NaN,False,NaN,5,2,NaN
3,Metal_LOW_09_Darkside.mp3,concert,clip_context_g1,5eff5f05b92981000a2aed73,clip_context_g1_5eff5f05b92981000a2aed73_02059...,2025-07-01_10h45.13.126,y,"very heavy rock, not for me. somewhere that i ...",5.0,5.0,...,United Kingdom,United Kingdom,English,False,NaN,False,NaN,5,2,NaN
4,Metal_MED_20_Welcome_to_the_Family.mp3,video game,clip_context_g1,5eff5f05b92981000a2aed73,clip_context_g1_5eff5f05b92981000a2aed73_02059...,2025-07-01_10h45.13.126,y,"very charged, maybe you've won something or wo...",5.0,4.0,...,United Kingdom,United Kingdom,English,False,NaN,False,NaN,5,2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2554,Metal_MED_20_Welcome_to_the_Family.mp3,concert,clip_context_g4,6824fc226d9b4777f8695cf0,clip_context_g4_PROLIFIC_PID_992291.csv,2025-07-01_06h12.29.151,y,A rock band made up of teenage white kids play...,5.0,5.0,...,United States,United States,English,False,NaN,False,NaN,3,2,none
2555,80s_LOW_02_Breaking_Away.mp3,concert,clip_context_g4,6824fc226d9b4777f8695cf0,clip_context_g4_PROLIFIC_PID_992291.csv,2025-07-01_06h12.29.151,y,People in a ballroom in elegant dresses slow d...,5.0,5.0,...,United States,United States,English,False,NaN,False,NaN,3,2,none
2557,Jazz_MED_02_I_Guess_Ill_Hang_My_Tears_Out_To_D...,video game,clip_context_g4,6824fc226d9b4777f8695cf0,clip_context_g4_PROLIFIC_PID_992291.csv,2025-07-01_06h12.29.151,y,I imagined a jazz festival and old men on stag...,5.0,5.0,...,United States,United States,English,False,NaN,False,NaN,3,2,none
2558,Electronic_MED_20_The_Distance.mp3,movie,clip_context_g4,6824fc226d9b4777f8695cf0,clip_context_g4_PROLIFIC_PID_992291.csv,2025-07-01_06h12.29.151,y,I imagined a documentary mostly about fun fact...,5.0,5.0,...,United States,United States,English,False,NaN,False,NaN,3,2,none


Index(['clip_name', 'context_word', 'expName', 'PROLIFIC_PID', 'File_ID',
       'date', 'response_thought_or_not.keys', 'descr_THOUGHT.text',
       'rating_music_prompted.response', 'rating_spontaneity.response',
       'rating_novelty.response', 'input_NOT.text',
       'rating_familiarity.response', 'rating_enjoyment.response',
       'demographics.headphones', 'demographics.age', 'demographics.gender',
       'demographics.livingCountry', 'demographics.birthCountry',
       'demographics.nativeLanguage', 'demographics.otherLanguage',
       'demographics.otherLanguageText', 'demographics.hearingImpariments',
       'demographics.hearingImpairmentsText', 'demographics.education',
       'demographics.musicianIdentification', 'demographics.feedback'],
      dtype='object')


We're going to combine the clip and context values into an additional column ("clip_context_PAIR") that we can use in functions later.

We can also drop the columns that where only relevant to distinguishing have- and have-not thoughts.

In [6]:
def create_clip_context_pair(row):
    if 'bar' in row['context_word']:
        return 'BAR-' + row['clip_name']
    elif 'video game' in row['context_word']:
        return 'VIDEOGAME-' + row['clip_name']
    elif 'concert' in row['context_word']:
        return 'CONCERT-' + row['clip_name']
    elif 'movie' in row['context_word']:
        return 'MOVIE-' + row['clip_name']
    else:
        return row['clip_name'] # Default to just the clip name if no match

dataMET['clip_context_PAIR'] = dataMET.apply(create_clip_context_pair, axis=1)

dataMET.drop(columns = ['response_thought_or_not.keys', 'input_NOT.text'],
             inplace=True)

# Check the dataframe by a quick re-view :)
display(dataMET)
print(dataMET.columns)

# Saving a .csv for the option to open and look at the full dataframe
dataMET.to_csv('/content/context-framed-listening/dataMET.csv', encoding='utf-8')

,clip_name,context_word,expName,PROLIFIC_PID,File_ID,date,descr_THOUGHT.text,rating_music_prompted.response,rating_spontaneity.response,rating_novelty.response,...,demographics.birthCountry,demographics.nativeLanguage,demographics.otherLanguage,demographics.otherLanguageText,demographics.hearingImpariments,demographics.hearingImpairmentsText,demographics.education,demographics.musicianIdentification,demographics.feedback,clip_context_PAIR
0,80s_LOW_02_Breaking_Away.mp3,bar,clip_context_g1,5eff5f05b92981000a2aed73,clip_context_g1_5eff5f05b92981000a2aed73_02059...,2025-07-01_10h45.13.126,"kind of sad, melancholy. not happy or upbeat. ...",5.0,4.0,3.0,...,United Kingdom,English,False,NaN,False,NaN,5,2,NaN,BAR-80s_LOW_02_Breaking_Away.mp3
1,Jazz_MED_07_Turiya_and_Ramakrishna.mp3,video game,clip_context_g1,5eff5f05b92981000a2aed73,clip_context_g1_5eff5f05b92981000a2aed73_02059...,2025-07-01_10h45.13.126,it did not sound like a video game. if anythin...,5.0,5.0,2.0,...,United Kingdom,English,False,NaN,False,NaN,5,2,NaN,VIDEOGAME-Jazz_MED_07_Turiya_and_Ramakrishna.mp3
2,80s_MED_08_After_Tonight.mp3,video game,clip_context_g1,5eff5f05b92981000a2aed73,clip_context_g1_5eff5f05b92981000a2aed73_02059...,2025-07-01_10h45.13.126,"overly upbeat. no real emotions, peppy. too mu...",5.0,4.0,4.0,...,United Kingdom,English,False,NaN,False,NaN,5,2,NaN,VIDEOGAME-80s_MED_08_After_Tonight.mp3
3,Metal_LOW_09_Darkside.mp3,concert,clip_context_g1,5eff5f05b92981000a2aed73,clip_context_g1_5eff5f05b92981000a2aed73_02059...,2025-07-01_10h45.13.126,"very heavy rock, not for me. somewhere that i ...",5.0,5.0,3.0,...,United Kingdom,English,False,NaN,False,NaN,5,2,NaN,CONCERT-Metal_LOW_09_Darkside.mp3
4,Metal_MED_20_Welcome_to_the_Family.mp3,video game,clip_context_g1,5eff5f05b92981000a2aed73,clip_context_g1_5eff5f05b92981000a2aed73_02059...,2025-07-01_10h45.13.126,"very charged, maybe you've won something or wo...",5.0,4.0,2.0,...,United Kingdom,English,False,NaN,False,NaN,5,2,NaN,VIDEOGAME-Metal_MED_20_Welcome_to_the_Family.mp3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2554,Metal_MED_20_Welcome_to_the_Family.mp3,concert,clip_context_g4,6824fc226d9b4777f8695cf0,clip_context_g4_PROLIFIC_PID_992291.csv,2025-07-01_06h12.29.151,A rock band made up of teenage white kids play...,5.0,5.0,4.0,...,United States,English,False,NaN,False,NaN,3,2,none,CONCERT-Metal_MED_20_Welcome_to_the_Family.mp3
2555,80s_LOW_02_Breaking_Away.mp3,concert,clip_context_g4,6824fc226d9b4777f8695cf0,clip_context_g4_PROLIFIC_PID_992291.csv,2025-07-01_06h12.29.151,People in a ballroom in elegant dresses slow d...,5.0,5.0,5.0,...,United States,English,False,NaN,False,NaN,3,2,none,CONCERT-80s_LOW_02_Breaking_Away.mp3
2557,Jazz_MED_02_I_Guess_Ill_Hang_My_Tears_Out_To_D...,video game,clip_context_g4,6824fc226d9b4777f8695cf0,clip_context_g4_PROLIFIC_PID_992291.csv,2025-07-01_06h12.29.151,I imagined a jazz festival and old men on stag...,5.0,5.0,4.0,...,United States,English,False,NaN,False,NaN,3,2,none,VIDEOGAME-Jazz_MED_02_I_Guess_Ill_Hang_My_Tear...
2558,Electronic_MED_20_The_Distance.mp3,movie,clip_context_g4,6824fc226d9b4777f8695cf0,clip_context_g4_PROLIFIC_PID_992291.csv,2025-07-01_06h12.29.151,I imagined a documentary mostly about fun fact...,5.0,5.0,5.0,...,United States,English,False,NaN,False,NaN,3,2,none,MOVIE-Electronic_MED_20_The_Distance.mp3


Index(['clip_name', 'context_word', 'expName', 'PROLIFIC_PID', 'File_ID',
       'date', 'descr_THOUGHT.text', 'rating_music_prompted.response',
       'rating_spontaneity.response', 'rating_novelty.response',
       'rating_familiarity.response', 'rating_enjoyment.response',
       'demographics.headphones', 'demographics.age', 'demographics.gender',
       'demographics.livingCountry', 'demographics.birthCountry',
       'demographics.nativeLanguage', 'demographics.otherLanguage',
       'demographics.otherLanguageText', 'demographics.hearingImpariments',
       'demographics.hearingImpairmentsText', 'demographics.education',
       'demographics.musicianIdentification', 'demographics.feedback',
       'clip_context_PAIR'],
      dtype='object')


---
## Descriptive Statistics

Below we are going to run some basic descriptive statistics on the clip-context pairings.


We're going to create a dataframe that includes summary info about each clip-context stimuli pairing including:

* Number of participants that reported METs while listening
* Mean ratings of MET "prompting-power", spontanteity, and novelty, and clip familiarity and enjoyment

In [7]:
columns = dataMET.columns.tolist()[1:-1]

# Drop these following columns so they don't aggregate by clip-context grouping:
drop = ['clip_name', 'context_word', 'expName', 'File_ID', 'date',
        'descr_THOUGHT.text', 'demographics.headphones', 'demographics.age',
        'demographics.gender','demographics.livingCountry',
        'demographics.birthCountry', 'demographics.nativeLanguage',
        'demographics.otherLanguage', 'demographics.otherLanguageText',
        'demographics.hearingImpariments', 'demographics.hearingImpairmentsText',
        'demographics.education','demographics.musicianIdentification',
        'demographics.feedback']

# Setting up an aggregate function collector
agg_fun = {}

# As we dropped trials without METs, we can just sum participants for MET occurrence
agg_fun['PROLIFIC_PID'] = 'count'

# Taking the mean of all columns except participant IDs and dropped columns
for col in columns:
    if col not in drop and col != 'PROLIFIC_PID':
        agg_fun[col] = 'mean'

# Group the dataframe by clip-context pairing, then run the aggregate functions created above
clipContextDescrStats = dataMET.groupby('clip_context_PAIR').agg(agg_fun)
display(clipContextDescrStats)

# Saving a .csv for the option to open and look at the full dataframe
clipContextDescrStats.to_csv('/content/context-framed-listening/clipContextDescrStats.csv', encoding='utf-8')

,PROLIFIC_PID,rating_music_prompted.response,rating_spontaneity.response,rating_novelty.response,rating_familiarity.response,rating_enjoyment.response
clip_context_PAIR,,,,,,
BAR-80s_LOW_02_Breaking_Away.mp3,34,4.235294,3.882353,3.382353,2.294118,3.705882
BAR-80s_LOW_06_Summer_Beach.mp3,29,4.448276,4.103448,3.482759,2.586207,3.793103
BAR-80s_MED_08_After_Tonight.mp3,29,4.275862,3.862069,2.965517,2.000000,3.310345
BAR-80s_MED_13_Your_Love_Drives_Me_Crazy.mp3,25,4.160000,3.840000,2.360000,2.280000,3.600000
BAR-Electronic_LOW_09_Expansion.mp3,27,4.592593,3.962963,2.259259,2.000000,3.259259
...,...,...,...,...,...,...
VIDEOGAME-Jazz_MED_07_Turiya_and_Ramakrishna.mp3,35,4.428571,4.285714,2.685714,2.457143,3.742857
VIDEOGAME-Metal_LOW_09_Darkside.mp3,34,4.558824,3.794118,2.411765,2.029412,3.235294
VIDEOGAME-Metal_LOW_14_Viaje_Por_Existir.mp3,26,4.307692,3.923077,2.423077,2.230769,3.346154


We can check here what the minimum and maximum reported MET occurences were to each clip-context stimuli pairing.

In [8]:
mostMETs_ccpair = clipContextDescrStats['PROLIFIC_PID'].idxmax()
leastMETs_ccpair = clipContextDescrStats['PROLIFIC_PID'].idxmin()
mostMETs_value = clipContextDescrStats['PROLIFIC_PID'].max()
leastMETs_value = clipContextDescrStats['PROLIFIC_PID'].min()

print(f"Clip-context pair with the most reported METs: {mostMETs_ccpair} ({mostMETs_value})")
print(f"Clip-context pair with the least reported METs: {leastMETs_ccpair} ({leastMETs_value})")

Clip-context pair with the most reported METs: BAR-Electronic_LOW_14_Tape.mp3 (35)
Clip-context pair with the least reported METs: BAR-80s_MED_13_Your_Love_Drives_Me_Crazy.mp3 (25)




---



---

## Text Preprocessing

This next section goes through some text cleaning necessary to start feeding these METs into our NLP models.

Run the below cell to intall and import some other necessary packages:

In [18]:
!pip install pyspellchecker

import re
from collections import defaultdict

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

import nltk
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('stopwords')

from nltk import word_tokenize, pos_tag

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Remove punctuation and special characters:

In [19]:
def clean_text(text):
    if isinstance(text, str):
        text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
        return text.lower()
    return text

dataMET['cleaned_MET_descr'] = dataMET['descr_THOUGHT.text'].apply(clean_text)
display(dataMET[['descr_THOUGHT.text', 'cleaned_MET_descr']].head())

,descr_THOUGHT.text,cleaned_MET_descr
0,"kind of sad, melancholy. not happy or upbeat. ...",kind of sad melancholy not happy or upbeat emo...
1,it did not sound like a video game. if anythin...,it did not sound like a video game if anything...
2,"overly upbeat. no real emotions, peppy. too mu...",overly upbeat no real emotions peppy too much
3,"very heavy rock, not for me. somewhere that i ...",very heavy rock not for me somewhere that i do...
4,"very charged, maybe you've won something or wo...",very charged maybe youve won something or won ...


### Stop Words

This next cell removes "stop words" from the qualitative data.

There are some automatically loaded-in stop words from our imported packages including largely uninteresting words such as articles, conjunctions, prepositions, pronouns, and common verbs.

This improves the efficiency and accuracy of NLP analyses by ensuring the focus is on significant terms and their patterns (i.e. it's not particularly interesting to know the pattern of "the", "and", "in", "it", "is" across the dataset).


We are also going to set some **custom stop words** that aren't giving us any unique meaning from these METs that we are intereseted in analysing. This includes naming the genre of music they heard, the context cues provided, and naming the way in which they experienced the MET (not everyone specifies or is able to identify _how_ they experienced their MET, so this can be a misnomer / misleading in analyses).


**NB:** you can come back to this cell to add other words and re-run analyses if you think other insignificant words are obscuring the data or if too much is being taken out :)

In [20]:
from spellchecker import SpellChecker

customStopWords = ['music', 'song', 'songs', 'excerpt', 'excerpts', 'piece', 'pieces', 'clip', 'clips',
'nineteen', '50s', '1950s', 'fifties', '50', '1950', 'fifty', '60s', '1960s', 'sixties', '60', '1960', 'sixty', '70s', '1970s', 'seventies', '70', '1970', 'seventy', '80s', '1980s', 'eighties', '80', '1980', 'eighty', '90s', '1990s', 'nineties', '90', '1990', 'ninety', '00s', '2000s', 'noughties', '2000', 'y2k',
'ambient', 'classical', 'electronic', 'funk', 'jazz', 'metal', 'rock', 'pop',
'bar', 'club', 'concert', 'film', 'movie', 'videogame', 'video', 'game',
'think', 'thinks', 'thought', 'thinking',
'imagine', 'imagines', 'imagined', 'imagining',
'image', 'images', 'imaged', 'imaging', 'visualise', 'visualises', 'visualised', 'visualising', 'visualize', 'visualizes', 'visualized', 'visualizing', 'picture', 'pictures', 'pictured', 'picturing',
'scene', 'scenes', 'story', 'stories',
'memory', 'memories', 'reminder', 'reminders', 'remind', 'reminds', 'reminded', 'reminding', 'remember', 'remembers', 'remembered', 'remembering', 'reminiscent', 'reminisce', 'reminisces', 'reminisced', 'reminiscing',
'make', 'makes', 'made', 'making', 'sound', 'sounds', 'sounded', 'sounding',
"'s", "n't", 'to' , 'of', 'and', 'like', 'around', 'also']

# Add NLTK stop words to custom stop words
nltk_stop_words = set(nltk.corpus.stopwords.words('english'))
all_stop_words = set(customStopWords).union(nltk_stop_words)

lemmatizer = nltk.stem.WordNetLemmatizer()
wn = nltk.corpus.wordnet

tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

spell = SpellChecker()

def preprocess_text(text):
    if isinstance(text, str):
        # Clean punctuation and convert to lower case (already done in previous step, but good to keep here for a complete function)
        text = re.sub(r'[^a-zA-Z0-9\s]', '', text.lower())

        # Tokenize
        tokens = word_tokenize(text)

        # Spelling correction and lemmatization
        lemmatized_tokens = []
        for word in tokens:
            # Spelling correction
            corrected_word = spell.correction(word) if spell.correction(word) else word

            # Lemmatization
            tag = pos_tag([corrected_word])[0][1][0].upper()
            lemma = lemmatizer.lemmatize(corrected_word, tag_map[tag])
            lemmatized_tokens.append(lemma)

        # Remove stop words
        filtered_tokens = [word for word in lemmatized_tokens if word not in all_stop_words]

        return " ".join(filtered_tokens)
    return text

dataMET['preprocessed_MET_descr'] = dataMET['descr_THOUGHT.text'].apply(preprocess_text)

display(dataMET[['descr_THOUGHT.text', 'preprocessed_MET_descr']].head())

KeyboardInterrupt: 